In [1]:
# train_translit.py
import os
import json
import math
import random
from collections import defaultdict
from copy import deepcopy
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, TensorDataset, random_split

from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
# Define the base path to authors
base_path = '/kaggle/input/urdu-ghazal-dataset/dataset/dataset/dataset'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [3]:
# Example experiments (at least 3). You can expand this list.
experiments = [
    {
        "name": "exp_small",
        "embed_dim": 128,
        "hidden_dim": 256,
        "enc_layers": 2,   # BiLSTM layers
        "dec_layers": 4,
        "dropout": 0.1,
        "lr": 1e-3,
        "batch_size": 32,
        "num_epochs": 10,
    },
    {
        "name": "exp_medium",
        "embed_dim": 256,
        "hidden_dim": 512,
        "enc_layers": 2,
        "dec_layers": 4,
        "dropout": 0.3,
        "lr": 5e-4,
        "batch_size": 32,
        "num_epochs": 8,
    },
    {
        "name": "exp_large",
        "embed_dim": 256,
        "hidden_dim": 512,
        "enc_layers": 1,
        "dec_layers": 4,
        "dropout": 0.3,
        "lr": 1e-4,
        "batch_size": 64,
        "num_epochs": 6,
    }
]

In [4]:
# ---------------------------
# ----- Utilities / Norm -----
# ---------------------------
special_tokens = ["[PAD]", "[SOS]", "[EOS]", "[UNK]"]
PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN = special_tokens

def normalize_urdu(text: str) -> str:
    """Basic normalization for Urdu: trim, collapse spaces, remove weird control chars.
       Extend as needed (e.g., normalize alef/ye/hah variations)."""
    t = text.strip()
    t = " ".join(t.split())  # collapse whitespace
    # remove control characters
    t = "".join(ch for ch in t if ord(ch) >= 32 or ch in "\n\t")
    return t

def char_tokenize(text: str) -> List[str]:
    return list(text)

In [5]:
# ---------------------------
# ----- Dataset Loading -----
# ---------------------------
def load_parallel_pairs(base_path: str) -> Tuple[List[str], List[str], dict]:
    urdu_texts = []
    roman_texts = []
    authors = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    # dataset audit
    author_counts = {}
    for author in authors:
        ur_path = os.path.join(base_path, author, 'ur')
        en_path = os.path.join(base_path, author, 'en')
        if os.path.exists(ur_path) and os.path.exists(en_path):
            ur_files = os.listdir(ur_path)
            en_files = os.listdir(en_path)
            common_files = set(ur_files) & set(en_files)
            author_counts[author] = len(common_files)
            for file_name in common_files:
                ur_file = os.path.join(ur_path, file_name)
                en_file = os.path.join(en_path, file_name)
                with open(ur_file, 'r', encoding='utf-8') as f:
                    ur = normalize_urdu(f.read())
                with open(en_file, 'r', encoding='utf-8') as f:
                    ro = f.read().strip()
                if ur and ro:
                    urdu_texts.append(ur)
                    roman_texts.append(ro)
    print(f"Loaded {len(urdu_texts)} parallel pairs from {len(author_counts)} authors.")
    return urdu_texts, roman_texts, author_counts


In [6]:
# ---------------------------
# ------- Vocab build -------
# ---------------------------
def build_char_vocab(texts: List[str]):
    token2id = {tok: idx for idx, tok in enumerate(special_tokens)}
    id2token = {idx: tok for tok, idx in token2id.items()}
    for t in texts:
        for ch in char_tokenize(t):
            if ch not in token2id:
                idx = len(token2id)
                token2id[ch] = idx
                id2token[idx] = ch
    return token2id, id2token

def encode_text(text: str, token2id: dict) -> List[int]:
    return [token2id.get(ch, token2id[UNK_TOKEN]) for ch in char_tokenize(text)]


In [7]:
# ---------------------------
# ----- Seq2Seq Model -------
# ---------------------------
class BiLSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, enc_hidden_dim, enc_layers, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        # enc_hidden_dim is per-direction hidden size
        self.enc_hidden_dim = enc_hidden_dim
        self.lstm = nn.LSTM(embed_dim, enc_hidden_dim, num_layers=enc_layers,
                            batch_first=True, bidirectional=True, dropout=dropout if enc_layers > 1 else 0.0)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        # src: (B, T)
        embedded = self.dropout(self.embedding(src))  # (B, T, E)
        # pack
        packed = pack_padded_sequence(embedded, src_lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (h_n, c_n) = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # (B, T, 2*H)
        # h_n: (num_layers * num_directions, B, H)
        return out, (h_n, c_n)

class DecoderLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, dec_hidden_dim, dec_layers, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, dec_hidden_dim, num_layers=dec_layers,
                            batch_first=True, dropout=dropout if dec_layers > 1 else 0.0)
        self.fc_out = nn.Linear(dec_hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_tokens, hidden):
        # input_tokens: (B, T) indices
        emb = self.dropout(self.embedding(input_tokens))  # (B, T, E)
        output, (h, c) = self.lstm(emb, hidden)  # (B, T, H)
        logits = self.fc_out(output)  # (B, T, V)
        return logits, (h, c)
        
class Seq2SeqBiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, enc_layers, dec_layers, dropout, pad_idx):
        super().__init__()
        assert hidden_dim % 2 == 0, "hidden_dim must be even for BiLSTM hidden split."
        enc_hidden_dim = hidden_dim // 2
        self.encoder = BiLSTMEncoder(vocab_size, embed_dim, enc_hidden_dim, enc_layers, dropout, pad_idx)
        self.decoder = DecoderLSTM(vocab_size, embed_dim, hidden_dim, dec_layers, dropout, pad_idx)

        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.hidden_dim = hidden_dim

        enc_state_size = enc_layers * 2 * enc_hidden_dim
        dec_state_size = dec_layers * hidden_dim
        self.h_proj = nn.Linear(enc_state_size, dec_state_size)
        self.c_proj = nn.Linear(enc_state_size, dec_state_size)

    def _reshape_encoder_states(self, h_n, c_n):
        B = h_n.size(1)
        enc_flat_h = h_n.transpose(0, 1).contiguous().view(B, -1)
        enc_flat_c = c_n.transpose(0, 1).contiguous().view(B, -1)
        dec_h_flat = self.h_proj(enc_flat_h)
        dec_c_flat = self.c_proj(enc_flat_c)
        dec_h = dec_h_flat.view(B, self.dec_layers, self.hidden_dim).transpose(0, 1).contiguous()
        dec_c = dec_c_flat.view(B, self.dec_layers, self.hidden_dim).transpose(0, 1).contiguous()
        return dec_h, dec_c

    def forward(self, src, src_lengths, tgt, debug=False):
        """
        Hard teacher forcing (100%): always feed ground-truth tokens during training.
        """
        enc_out, (h_n, c_n) = self.encoder(src, src_lengths)
        dec_h, dec_c = self._reshape_encoder_states(h_n, c_n)

        B, T_tgt = tgt.size()
        logits = []

        dec_input = tgt[:, 0].unsqueeze(1)  # start with <sos>
        hidden = (dec_h, dec_c)

        for t in range(1, T_tgt):
            logit_step, hidden = self.decode_step(dec_input, hidden)
            logits.append(logit_step.unsqueeze(1))

            # 🔑 hard teacher forcing: always use ground-truth
            next_input = tgt[:, t]
            dec_input = next_input.unsqueeze(1)

        logits = torch.cat(logits, dim=1)  # (B, T_tgt-1, V)

        # Debug: show first batch comparison
        if debug:
            pred_tokens = logits.argmax(-1)[0, :20].tolist()
            true_tokens = tgt[0, 1:21].tolist()
            print("DEBUG predicted tokens:", pred_tokens)
            print("DEBUG true tokens     :", true_tokens)

        return logits


    def encode_hidden(self, src, src_lengths):
        enc_out, (h_n, c_n) = self.encoder(src, src_lengths)
        dec_h, dec_c = self._reshape_encoder_states(h_n, c_n)
        return dec_h, dec_c

    def decode_step(self, input_token, hidden):
        # input_token: (B, 1)
        logits, hidden = self.decoder(input_token, hidden)  # pass indices, not embeddings
        return logits[:, -1, :], hidden


In [8]:
# ---------------------------
# ---- Metrics (BLEU/CER) --
# ---------------------------
def levenshtein(a: List[str], b: List[str]) -> int:
    # classic DP
    la, lb = len(a), len(b)
    if la == 0: return lb
    if lb == 0: return la
    dp = [list(range(lb+1))] + [[i+1] + [0]*lb for i in range(la)]
    for i in range(1, la+1):
        for j in range(1, lb+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(dp[i-1][j] +1, dp[i][j-1]+1, dp[i-1][j-1] + cost)
    return dp[la][lb]

def cer(pred: str, ref: str) -> float:
    # character error rate = edit distance / len(ref)
    ed = levenshtein(list(pred), list(ref))
    denom = max(1, len(ref))
    return ed / denom

def simple_bleu(cand: str, ref: str, max_n=4) -> float:
    # Simple corpus-level approximation for one sentence (BLEU-4 with geometric mean & BP)
    # Tokenize by characters (transliteration) or by whitespace? Use character-ngrams since transliteration is char-based.
    def ngram_counts(s, n):
        s = list(s)
        return defaultdict(int, {tuple(s[i:i+n]): sum(1 for j in range(len(s)-n+1) if tuple(s[j:j+n])==tuple(s[i:i+n])) for i in range(len(s)-n+1)}) if len(s)>=n else {}
    precisions = []
    for n in range(1, max_n+1):
        c_counts = ngram_counts(cand, n)
        r_counts = ngram_counts(ref, n)
        if not c_counts:
            precisions.append(0.0)
            continue
        # clipped count
        clipped = 0
        total = 0
        for ng, cnt in c_counts.items():
            clipped += min(cnt, r_counts.get(ng, 0))
            total += cnt
        precisions.append(clipped / total if total>0 else 0.0)
    # geometric mean
    if min(precisions) == 0:
        geo_mean = 0.0
    else:
        geo_mean = math.exp(sum(math.log(p) for p in precisions)/len(precisions))
    # brevity penalty
    len_c = len(cand)
    len_r = len(ref)
    bp = 1.0 if len_c > len_r else math.exp(1 - len_r / (len_c + 1e-9))
    return bp * geo_mean

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1, ignore_index=-100):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            mask = target != self.ignore_index
            true_dist[mask, target[mask]] = self.confidence
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))


In [9]:
# ---------------------------
# ----- Data preparation -----
# ---------------------------
def prepare_datasets(urdu_texts, roman_texts, test_size=0.5):
    # Provided: we need 50/25/25 train/val/test. We'll split stepwise:
    # First split 50% train, 50% temp. Then split temp into val/test (half each).
    # We use sklearn train_test_split for reproducibility.
    ur_train, ur_temp, ro_train, ro_temp = train_test_split(urdu_texts, roman_texts, test_size=0.5, random_state=42)
    ur_val, ur_test, ro_val, ro_test = train_test_split(ur_temp, ro_temp, test_size=0.5, random_state=42)
    return (ur_train, ro_train), (ur_val, ro_val), (ur_test, ro_test)

def collate_and_encode(batch_texts, token2id, add_special=True):
    # encode list of strings -> padded LongTensor, lengths
    tensors = []
    lengths = []
    for t in batch_texts:
        encoded = encode_text(t, token2id)
        if add_special:
            encoded = [token2id[SOS_TOKEN]] + encoded + [token2id[EOS_TOKEN]]
        tensors.append(torch.tensor(encoded, dtype=torch.long))
        lengths.append(len(encoded))
    padded = pad_sequence(tensors, batch_first=True, padding_value=token2id[PAD_TOKEN])
    lengths = torch.tensor(lengths, dtype=torch.long)
    return padded, lengths

In [10]:
# ---------------------------
# ----- Training / Eval -----
# ---------------------------
def compute_token_accuracy(preds: torch.Tensor, targets: torch.Tensor, pad_id: int) -> Tuple[int,int]:
    # preds: (B, T), targets: (B, T)
    mask = (targets != pad_id)
    correct = ((preds == targets) & mask).sum().item()
    total = mask.sum().item()
    return correct, total

def train_one_epoch(model, train_loader, optimizer, criterion, token2id, device):
    model.train()
    running_loss = 0.0
    running_correct = 0
    running_total = 0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_idx, (src_batch, src_lens, tgt_batch, tgt_lens) in loop:
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)
        src_lens = src_lens.to(device)

        optimizer.zero_grad()
        logits = model(src_batch, src_lens, tgt_batch)  # (B, T-1, V)
        V = logits.size(-1)
        loss = criterion(logits.reshape(-1, V), tgt_batch[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = logits.argmax(dim=-1)  # (B, T-1)
        correct, total = compute_token_accuracy(preds, tgt_batch[:, 1:], token2id[PAD_TOKEN])
        running_correct += correct
        running_total += total

        avg_loss = running_loss / (batch_idx + 1)
        avg_acc = running_correct / running_total if running_total>0 else 0.0
        loop.set_description(f"Train")
        loop.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{avg_acc:.4f}")

    return running_loss / len(train_loader), running_correct / running_total if running_total>0 else 0.0

def evaluate(model, data_loader, criterion, token2id, id2token, device, max_decoding_len=200):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_tokens = 0
    all_preds = []
    all_refs = []
    with torch.no_grad():
        for src_batch, src_lens, tgt_batch, tgt_lens in data_loader:
            src_batch = src_batch.to(device)
            tgt_batch = tgt_batch.to(device)
            src_lens = src_lens.to(device)

            logits = model(src_batch, src_lens, tgt_batch)
            V = logits.size(-1)
            loss = criterion(logits.reshape(-1, V), tgt_batch[:, 1:].reshape(-1))
            total_loss += loss.item()

            preds = logits.argmax(dim=-1)
            correct, total = compute_token_accuracy(preds, tgt_batch[:, 1:], token2id[PAD_TOKEN])
            total_correct += correct
            total_tokens += total

            # For BLEU / CER, produce decoded strings via autoregressive decoding per sample
            B = src_batch.size(0)
            dec_h, dec_c = model.encode_hidden(src_batch, src_lens)
            hidden = (dec_h.to(device), dec_c.to(device))
            # start tokens
            input_token = torch.full((B,1), token2id[SOS_TOKEN], dtype=torch.long, device=device)
            seqs = [[] for _ in range(B)]
            finished = [False]*B
            for step in range(max_decoding_len):
                logits_step, hidden = model.decode_step(input_token, hidden)
                next_tokens = logits_step.argmax(dim=-1)  # (B)
                input_token = next_tokens.unsqueeze(1)
                for i in range(B):
                    tok = next_tokens[i].item()
                    if tok == token2id[EOS_TOKEN] or tok == token2id[PAD_TOKEN]:
                        finished[i] = True
                    else:
                        seqs[i].append(id2token.get(tok, UNK_TOKEN))
                if all(finished):
                    break
            # join preds and refs
            for i in range(B):
                pred_s = "".join(seqs[i])
                # target was tgt_batch ith, remove sos & eos & pads
                tgt_indices = tgt_batch[i].cpu().tolist()
                # strip SOS at 0 and anything after EOS
                tgt_chars = []
                for idx in tgt_indices[1:]:
                    if idx == token2id[EOS_TOKEN] or idx == token2id[PAD_TOKEN]:
                        break
                    tgt_chars.append(id2token.get(idx, UNK_TOKEN))
                ref_s = "".join(tgt_chars)
                all_preds.append(pred_s)
                all_refs.append(ref_s)

    avg_loss = total_loss / len(data_loader)
    acc = total_correct / total_tokens if total_tokens>0 else 0.0
    # compute BLEU & CER averages
    bleu_scores = [simple_bleu(p, r) for p,r in zip(all_preds, all_refs)]
    cer_scores = [cer(p, r) for p,r in zip(all_preds, all_refs)]
    avg_bleu = sum(bleu_scores)/len(bleu_scores) if bleu_scores else 0.0
    avg_cer = sum(cer_scores)/len(cer_scores) if cer_scores else 0.0

    return avg_loss, acc, avg_bleu, avg_cer, all_preds, all_refs

In [11]:
# ---------------------------
# ------ DataLoader prep ----
# ---------------------------
def make_dataloader_from_texts(ur_texts, ro_texts, token2id, batch_size, shuffle=True):
    # encode all texts into tensors with SOS/EOS
    src_tensors = []
    src_lens = []
    tgt_tensors = []
    tgt_lens = []
    for ur, ro in zip(ur_texts, ro_texts):
        src_enc = encode_text(ur, token2id)
        src_enc = [token2id[SOS_TOKEN]] + src_enc + [token2id[EOS_TOKEN]]
        tgt_enc = encode_text(ro, token2id)
        tgt_enc = [token2id[SOS_TOKEN]] + tgt_enc + [token2id[EOS_TOKEN]]
        src_tensors.append(torch.tensor(src_enc, dtype=torch.long))
        tgt_tensors.append(torch.tensor(tgt_enc, dtype=torch.long))
        src_lens.append(len(src_enc))
        tgt_lens.append(len(tgt_enc))
    src_padded = pad_sequence(src_tensors, batch_first=True, padding_value=token2id[PAD_TOKEN])
    tgt_padded = pad_sequence(tgt_tensors, batch_first=True, padding_value=token2id[PAD_TOKEN])
    src_lens_tensor = torch.tensor(src_lens, dtype=torch.long)
    tgt_lens_tensor = torch.tensor(tgt_lens, dtype=torch.long)
    dataset = TensorDataset(src_padded, src_lens_tensor, tgt_padded, tgt_lens_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader


In [12]:
# ---------------------------
# ------- Run experiments ----
# ---------------------------
def run_experiment(cfg, urdu_texts, roman_texts, author_counts):
    print(f"\n=== Running experiment: {cfg['name']} ===")
    # Prepare splits: 50/25/25
    (ur_train, ro_train), (ur_val, ro_val), (ur_test, ro_test) = prepare_datasets(urdu_texts, roman_texts)
    print(f"Train: {len(ur_train)} | Val: {len(ur_val)} | Test: {len(ur_test)}")

    # Build vocabulary on combined train + val + test (or only train?) — best practice: build on train
    token2id, id2token = build_char_vocab(ur_train + ro_train)
    # ensure special tokens exist
    for tok in special_tokens:
        if tok not in token2id:
            token2id[tok] = len(token2id)
            id2token[token2id[tok]] = tok

    pad_id = token2id[PAD_TOKEN]

    # DataLoaders
    train_loader = make_dataloader_from_texts(ur_train, ro_train, token2id, cfg["batch_size"], shuffle=True)
    val_loader = make_dataloader_from_texts(ur_val, ro_val, token2id, cfg["batch_size"], shuffle=False)
    test_loader = make_dataloader_from_texts(ur_test, ro_test, token2id, cfg["batch_size"], shuffle=False)

    # Model
    vocab_size = len(token2id)
    model = Seq2SeqBiLSTM(vocab_size=vocab_size,
                         embed_dim=cfg["embed_dim"],
                         hidden_dim=cfg["hidden_dim"],
                         enc_layers=cfg["enc_layers"],
                         dec_layers=cfg["dec_layers"],
                         dropout=cfg["dropout"],
                         pad_idx=pad_id).to(device)

    criterion = LabelSmoothingLoss(classes=vocab_size, smoothing=0.1, ignore_index=pad_id)
    optimizer = optim.Adam(model.parameters(), lr=cfg["lr"])

    # training
    history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[], "val_bleu":[], "val_cer":[]}
    for epoch in range(cfg["num_epochs"]):
        print(f"\nEpoch {epoch+1}/{cfg['num_epochs']}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, token2id, device)
        val_loss, val_acc, val_bleu, val_cer, val_preds, val_refs = evaluate(model, val_loader, criterion, token2id, id2token, device)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)
        history["val_bleu"].append(val_bleu)
        history["val_cer"].append(val_cer)
        print(f"Epoch {epoch+1} summary | Train loss {train_loss:.4f}, Train acc {train_acc:.4f} | Val loss {val_loss:.4f}, Val acc {val_acc:.4f}, Val BLEU {val_bleu:.4f}, Val CER {val_cer:.4f}")
        # show a few qualitative examples from validation
        print("Sample val examples (pred => ref):")
        for i in range(min(5, len(val_preds))):
            print(f"  {i+1}) {val_preds[i]}  =>  {val_refs[i]}")

    # final evaluation on test set
    test_loss, test_acc, test_bleu, test_cer, test_preds, test_refs = evaluate(model, test_loader, criterion, token2id, id2token, device)
    print("\n=== Final Test Results ===")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test BLEU: {test_bleu:.4f}, Test CER: {test_cer:.4f}")
    # show qualitative examples on test
    print("\nSample test examples (pred => ref):")
    for i in range(min(10, len(test_preds))):
        print(f"{i+1}) {test_preds[i]}  =>  {test_refs[i]}")
    # return history and final metrics
    result = {
        "name": cfg["name"],
        "history": history,
        "test_metrics": {
            "loss": test_loss,
            "acc": test_acc,
            "bleu": test_bleu,
            "cer": test_cer
        }
    }
    return result

In [13]:
# ---------------------------
# --------- Main -----------
# ---------------------------
if __name__ == "__main__":
    urdu_texts, roman_texts, author_counts = load_parallel_pairs(base_path)

    # If dataset small, you might want to reduce experiment settings.
    # Run experiments (loop through experiments list)
    results = []
    # For demo, we'll run only the first experiment to save time.
    # To run all: for cfg in experiments: results.append(run_experiment(cfg, urdu_texts, roman_texts, author_counts))
    # Here I'll run all (comment/uncomment as needed)
    for cfg in experiments:
        res = run_experiment(cfg, urdu_texts, roman_texts, author_counts)
        results.append(res)
        # optionally save per-experiment results to JSON
        with open(f"{cfg['name']}_results.json", "w", encoding="utf-8") as f:
            json.dump(res, f, ensure_ascii=False, indent=2)
    print("\nAll experiments finished.")

Loaded 1314 parallel pairs from 30 authors.

=== Running experiment: exp_small ===
Train: 657 | Val: 328 | Test: 329

Epoch 1/10


Epoch 1 summary | Train loss 1.5150, Train acc 0.1592 | Val loss 1.2541, Val acc 0.1758, Val BLEU 0.0000, Val CER 0.8308
Sample val examples (pred => ref):
  1) aaaa                                                                                                                                                                                                      =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 2 summary | Train loss 1.3992, Train acc 0.1757 | Val loss 1.2509, Val acc 0.1759, Val BLEU 0.0000, Val CER 0.8308
Sample val examples (pred => ref):
  1) aaaa                                                                                                                                                                                                      =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 3 summary | Train loss 1.3965, Train acc 0.1757 | Val loss 1.2425, Val acc 0.1788, Val BLEU 0.0000, Val CER 0.7606
Sample val examples (pred => ref):
  1) aaa   a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a  a a   =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 4 summary | Train loss 1.3564, Train acc 0.2514 | Val loss 1.1819, Val acc 0.2699, Val BLEU 0.0010, Val CER 0.7958
Sample val examples (pred => ref):
  1) aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa aa aaa   =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

...

Epoch 2 summary | Train loss 1.6506, Train acc 0.1758 | Val loss 1.3069, Val acc 0.1760, Val BLEU 0.0000, Val CER 0.8310
Sample val examples (pred => ref):
  1) aa                                                                                                                                                                                                        =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 3 summary | Train loss 1.4559, Train acc 0.1758 | Val loss 1.2643, Val acc 0.1758, Val BLEU 0.0000, Val CER 0.8308
Sample val examples (pred => ref):
  1) aaaa                                                                                                                                                                                                      =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 4 summary | Train loss 1.4201, Train acc 0.1758 | Val loss 1.2401, Val acc 0.1760, Val BLEU 0.0000, Val CER 0.8310
Sample val examples (pred => ref):
  1) aa                                                                                                                                                                                                        =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 5 summary | Train loss 1.3928, Train acc 0.1759 | Val loss 1.2372, Val acc 0.1760, Val BLEU 0.0000, Val CER 0.8309
Sample val examples (pred => ref):
  1) aaa                                                                                                                                                                                                       =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā

Epoch 6 summary | Train loss 1.4065, Train acc 0.1758 | Val loss 1.2366, Val acc 0.1760, Val BLEU 0.0000, Val CER 0.8310
Sample val examples (pred => ref):
  1) aa                                                                                                                                                                                                        =>  azāb-e-dīd meñ āñkheñ lahū lahū kar ke 
maiñ sharmsār huā terī justujū kar ke 
khañDar kī tah se burīda-badan saroñ ke sivā 
milā na kuchh bhī ḳhazānoñ kī aarzū kar ke 
sunā hai shahar meñ zaḳhmī diloñ kā mela hai 
chaleñge ham bhī magar pairahan rafū kar ke 
masāfat-e-shab-e-hijrāñ ke ba.ad bhed khulā 
havā dukhī hai charāġhoñ kī aabrū kar ke 
zamīñ kī pyaas usī ke lahū ko chaaT ga.ī 
vo ḳhush huā thā samundar ko aabjū kar ke 
ye kis ne ham se lahū kā ḳhirāj phir māñgā 
abhī to so.e the maqtal ko surḳh-rū kar ke 
julūs-e-ahl-e-vafā kis ke dar pe pahuñchā hai 
nishān-e-tauq-e-vafā zīnat-e-gulū kar ke 
ujaaḌ rut ko gulābī banā